I need to decide which songs I want to collect

In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import config
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

### FMA metadata needed for spotify scraping

Get track names, artist names, album names, where set subset = 'small'

In [23]:
tracks_file_raw = pd.read_csv('tracks.csv')

C:\Users\Mendel Oster\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
# retrieve the indices for the 'small' subset
small_indices = tracks_file_raw[tracks_file_raw['set.1'] == 'small'].index

In [72]:
subset_tracks_df = tracks_file_raw.iloc[small_indices]
subset_tracks_df.shape

(8000, 53)

In [124]:
# finding which columns I need
tracks_file_raw.loc[[0]].iloc[:,:53]

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title


In [119]:
# slicing the tracks file to find the artist name, album name, track name, and track bit rate
artists = subset_tracks_df['artist.12']
albums = subset_tracks_df['album.10']
tracks = subset_tracks_df['track.19']
tracks_br = subset_tracks_df['track'].astype(int)
print('There are {} nulls in the artists name column'.format(artists.isnull().sum()))
print('There are {} nulls in the album name column'.format(albums.isnull().sum()))
print('There are {} nulls in the track name column'.format(tracks.isnull().sum()))
print('There are {} nulls in the track bit rate column'.format(tracks_br.isnull().sum()))

There are 0 nulls in the artists name column
There are 0 nulls in the album name column
There are 0 nulls in the track name column
There are 0 nulls in the track bit rate column


Function to clean track names into one string all lowercase, underscore for space

In [120]:
def clean_tracks(tracks_series):
    '''
    Takes in a pandas series of track names and converts into clean format ex: 'example_song_fireside'
    '''
    pass

In [123]:
tracks.head()

2                   Food
4             This World
5                Freeway
17    Queen Of The Wires
18                  Ohio
Name: track.19, dtype: object

### Testing Spotipy to retrieve time signature of songs in FMA small dataset , from spotify

In [179]:
# To match the search query results from spotify and the tracks which I have from FMA
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [175]:
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id=config.SPOTIPY_CLIENT_ID, client_secret= config.SPOTIPY_CLIENT_SECRET))

Need to make the searh query to only bring up tracks with that artist name or album name

In [180]:
test_srch_q = spotify.search(q='album:gold artist:abba', type='album')

Function to set up search request for spotify

In [186]:
def search_q(track, artist=None, album=None, qtype='track'):
    '''
    This function takes in a track name, artist name, and album name and will format a query for Spotipy.search.
    The goal of the query is to find the qtype of the query given multiple conditions
    for example 'album:gold artist:abba', type='album' will return albums with
    'gold' in it as long as the artist contains 'abba'
    qtype can be changed to anything spotify allows 
    see https://developer.spotify.com/documentation/web-api/reference/search/search/
    '''
    if artist and album:
        return spotify.search(q = 'track:{t} artist:{ar} album:{al}'.format(t=track, ar= artist, al= album),type=qtype)
    if artist and not album:
        return spotify.search(q = 'track:{t} artist:{ar}'.format(t=track, ar= artist),type=qtype)
    if not artist and album:
        return spotify.search(q = 'track:{t} album:{al}'.format(t=track, al= album),type=qtype)
    if not artist and not album:
        return spotify.search(q = 'track:{t}'.format(t=track),type=qtype)

In [295]:
#  testing
s1 =   search_q(tracks.iloc[400], artists.iloc[400])

In [254]:
s2 = search_q(tracks.iloc[0],album=albums.iloc[0])

In [258]:
s2

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AFood+album%3AAWOL+-+A+Way+Of+Life&type=track&offset=0&limit=10',
  'items': [],
  'limit': 10,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [199]:
s1['tracks']['items'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

Function to test the search results using Fuzzywuzzy

In [208]:
def match_results(query_result, track, artist=None, album=None):
    '''
    This function uses fuzzywuzzy to measure the similarity of the search query results and the tracks needed
    query result should be json format direct result from the function 'search_q' above
    returns a dictionary like so {'track': result, 'artist': result, 'album': result}
    note: the search_q function must be used with q_type = 'track' for this function to work
    '''
    t = query_result['tracks']['items'][0]['name']
    t_val = fuzz.ratio(t, track)
    if artist:
        ar = query_result['tracks']['items'][0]['artists'][0]['name']
        ar_val = fuzz.ratio(ar, artist)
    if album:
        al = query_result['tracks']['items'][0]['album']['name']
        al_val = fuzz.ratio(al, album)

    if artist and album:
        return {t: t_val, ar: ar_val, al: al_val}
    if artist and not album:
        return {t: t_val, ar: ar_val}
    if not artist and album:
        return {t: t_val, al: al_val}
    if not artist and not album:
        return {t: t_val}

In [219]:
print(s1['tracks']['items'][0]['album']['name'], ':',albums.iloc[400])
print(s1['tracks']['items'][0]['artists'][0]['name'],':', artists.iloc[400])
print(s1['tracks']['items'][0]['name'],':', tracks.iloc[400])

Crooked : Speedbath (working)
Kristin Hersh : Kristin Hersh
Mississippi Kite : Mississippi Kite


In [215]:
print(match_results(s1, tracks.iloc[400]))
print(match_results(s1, tracks.iloc[400], artist = artists.iloc[400]))
print(match_results(s1, tracks.iloc[400], album = albums.iloc[400]))
print(match_results(s1, tracks.iloc[400], artist = artists.iloc[400], album = albums.iloc[400]))

{'Mississippi Kite': 100}
{'Mississippi Kite': 100, 'Kristin Hersh': 100}
{'Mississippi Kite': 100, 'Crooked': 15}
{'Mississippi Kite': 100, 'Kristin Hersh': 100, 'Crooked': 15}


In [234]:
def measure_results(match_results, thresh=66):
    '''
    This function takes in a dictionary of fuzz match results see function 'match_reults'
    returns a boolean if the match is successful 
    default threshold is 66
    '''
    result = np.mean(list(match_results.values()))
    if result >= thresh:
        return True
    else:
        return False

In [235]:
measure_results(match_results(s1, tracks.iloc[400], artist = artists.iloc[400], album = albums.iloc[400]))

True

### Loop through tracks to find which search method returns the most matches

In [296]:
# stores (track index, returned query json) - should be len(8000)
store_S = []
# stores track index & match result for queries with a result like so 
# (400, {'Mississippi Kite': 100, 'Kristin Hersh': 100, 'Crooked': 15})
store_M = []
# stores track index & boolean like so (400,True)
store_B = []
# stores tuple of track index and its spotify ID - only for those who passed the test in B
track_spot_id = []
for i, song in tracks.items():
    
#     set variables to all have same index
    track = tracks.loc[i]
    artist = artists.loc[i]
    album = albums.loc[i]
       
#     Spotipy makes a search query in spotify, returns json
    S = search_q(track=track, artist=artist, album=album)
    store_S.append((i, S))
#     if search query returns empty list, move along
    if not S['tracks']['items']:
        continue
        
    M = match_results(S,track=track,artist=artist,album=album)
    B = measure_results(M)
    
    store_M.append((i, M))
    store_B.append((i, B))
    
#     if the search results don't pass the test for accuracy, move along
    if not B:
        continue
    
#     retrieve spotify id for song
    spot_ID = S['tracks']['items'][0]['id']
    track_spot_id.append((i, spot_ID))

once I have a set of jsons to work with I will retrieve the track ID like so
search_q['tracks]['items'][0]['id']
which wil then be put into spotiy.audio_analysis(track_id)



In [243]:
s1_analysis = spotify.audio_analysis(s1['tracks']['items'][0]['id'])

In [247]:
s1_analysis.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])

In [250]:
s1_analysis['track']['time_signature']

4

In [252]:
s1_analysis['track']['time_signature_confidence']

0.92

In [246]:
s1_analysis['beats']

[{'start': 0.1475, 'duration': 0.46129, 'confidence': 0.98},
 {'start': 0.60879, 'duration': 0.4603, 'confidence': 0.685},
 {'start': 1.0691, 'duration': 0.46558, 'confidence': 0.218},
 {'start': 1.53468, 'duration': 0.47882, 'confidence': 0.262},
 {'start': 2.0135, 'duration': 0.47536, 'confidence': 0.318},
 {'start': 2.48886, 'duration': 0.47364, 'confidence': 0.282},
 {'start': 2.9625, 'duration': 0.47439, 'confidence': 0.338},
 {'start': 3.43688, 'duration': 0.47617, 'confidence': 0.235},
 {'start': 3.91305, 'duration': 0.4736, 'confidence': 0.106},
 {'start': 4.38665, 'duration': 0.46682, 'confidence': 0.327},
 {'start': 4.85347, 'duration': 0.46702, 'confidence': 0.247},
 {'start': 5.32049, 'duration': 0.47059, 'confidence': 0.147},
 {'start': 5.79107, 'duration': 0.47594, 'confidence': 0.21},
 {'start': 6.26701, 'duration': 0.48064, 'confidence': 0.24},
 {'start': 6.74765, 'duration': 0.48126, 'confidence': 0.233},
 {'start': 7.22891, 'duration': 0.4838, 'confidence': 0.196},
 {

In [ ]:
# be sure to avergae the fuzz values ad set a threshold for acceptance

# for loop through tracks and scrape search queries for 
# matching tracks alone
# track &artist & album
# track & artist
# track & album
# see which one gets the best results 

# then retrieve the track ID and from there get anaysis data etc..

In [188]:
# if passes the test, function to retrieve track ID then need to extract analysis > retrieve time signature yipee

#### Trying regular spotify API

In [151]:
# may need to re run this every hour - spotify shuts puts the access on a timer
import requests

auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(auth_url, {
    'grant_type': 'client_credentials',
    'client_id': config.SPOTIPY_CLIENT_ID,
    'client_secret': config.SPOTIPY_CLIENT_SECRET
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
spot_headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [171]:
search_base_url = 'https://api.spotify.com/v1/search'
r1 = requests.get(search_base_url, headers = spot_headers, params={'q':'album:gold artist:abba', 'type': 'album'})

Dontforget to set the smaple rate to none in order to mantain the original soundtrack sample rate